<div class="alert alert-block alert-success">
    
__Lab activity: Stateless transformation__
    
1. Create a Ray Dataset from the iris data in `s3://anyscale-materials/data/iris.csv`
1. Create a "sum of features" transformation that calculates the sum of the Sepal Length, Sepal Width, Petal Length, and Petal Width features for the records
    1. Design this transformation to take a Ray Dataset *batch* of records
    1. Return the records without the ID column but with an additional column called "sum"
    1. Hint: you do not need to use NumPy, but the calculation may be easier/simpler to code using NumPy vectorized operations with the records in the batch
</div>


In [ ]:
import ray

In [ ]:
ds = ray.data.read_csv('s3://anyscale-materials/data/iris.csv')

In [ ]:
def sum_of_features(batch):
    sep_len, sep_wid, pet_len, pet_wid = batch['SepalLengthCm'], batch['SepalWidthCm'], batch['PetalLengthCm'], batch['PetalWidthCm']
    sums = [ sep_len[i] + sep_wid[i] + pet_len[i] + pet_wid[i] for i in range(len(sep_len)) ]
    batch['sum'] = sums
    del batch['Id']
    return batch

In [ ]:
ds.map_batches(sum_of_features).show()

In [ ]:
def sum_of_features_vec(batch):
    batch['sum'] = batch['SepalLengthCm'] + batch['SepalWidthCm'] + batch['PetalLengthCm'] + batch['PetalWidthCm']
    del batch['Id']
    return batch

ds.map_batches(sum_of_features).show()

<div class="alert alert-block alert-warning">
    
__Lab activity: Stateful transformation__
    
1. Create a Ray Dataset from the iris data in `s3://anyscale-materials/data/iris.csv`
1. Create an class that makes predictions on iris records using these steps:
    1. in the class constructor, create an instance of the following "model" class:
        ```python

          class SillyModel():

              def predict(self, petal_length):
                  return petal_length + 0.42


        ```
    1. in the `__call__` method of the actor class
        1. take a batch of records
        1. create predictions for each record in the batch using the model instance
            1. Hint: the code may be simpler using NumPy vectorized operations
        1. add the predictions to the record batch
        1. return the new, augmented batch
1. Use that class to perform batch inference on the dataset using actors
</div>

In [ ]:
ds = ray.data.read_csv('s3://anyscale-materials/data/iris.csv')

In [ ]:
class SillyModel():
    def predict(self, petal_length):
        return petal_length + 0.42

In [ ]:
class MyBatchPredictor():
    def __init__(self):
        self.model = SillyModel()
        
    def __call__(self, batch):
        batch['predictions'] = self.model.predict(batch['PetalLengthCm'])
        return batch

In [ ]:
ds.map_batches(MyBatchPredictor, concurrency=3).show()